# Trabalho 1

11218813 Bruna Magrini da Cruz<br>
11218809 Flavio Salles<br>
11219004 Yasmin Osajima de Araujo

# Bibliotecas

In [384]:
%pip install glfw
%pip install PyOpenGL

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [385]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math

# Janela

In [386]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(1020, 900, "Trabalho 1 - Jardim do ICMC", None, None)
glfw.make_context_current(window)

# GLSL

### `GLSL` (*OpenGL Shading Language*)
Aqui veremos nosso primeiro código `GLSL`.

É uma linguagem de *shading* de alto nível baseada na linguagem de programação `C`.

Estamos escrevendo código `GLSL` como *strings* de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Aprenderemos `GLSL` conforme a necessidade do curso. Usaremos uma versão do `GLSL` mais antiga, compatível com muitos dispositivos.
### `GLSL` para *Vertex Shader*
No Pipeline programável, podemos interagir com *Vertex Shaders*.

No código abaixo, estamos fazendo o seguinte:
* Definindo uma variável chamada `position` do tipo `vec2`.
* Definindo uma variável chamada `mat_transformation` do tipo `mat4` (matriz $4\times4$).
* Usamos `vec2`, pois nosso programa (na CPU) enviará apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (`vec3`) e até mesmo quatro coordenadas (`vec4`).
* `void main()` é o ponto de entrada do nosso programa (função principal).
* `gl_Position` é uma variável especial do `GLSL`. Variáveis que começam com `gl_` são desse tipo. Neste caso, determina a posição de um vértice. Observe que todo vértice tem $4$ coordenadas, por isso combinamos nossa variável `vec2` com uma variável `vec4`. Além disso, modificamos nosso vetor com base em uma matriz de transformação, conforme estudado na Aula 4.

In [387]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

### `GLSL` para *Fragment Shader*
No Pipeline programável, podemos interagir com *Fragment Shaders*.

No código abaixo, estamos fazendo o seguinte:
* `void main()` é o ponto de entrada do nosso programa (função principal).
* `gl_FragColor` é uma variável especial do `GLSL`. Variáveis que começam com `gl_` são desse tipo. Neste caso, determina a cor de um fragmento.

### Possibilitando modificar a cor
Nos exemplos anteriores, a variável `gl_FragColor` estava definida de forma fixa (com cor $R=0$, $G=0$, $B=0$).

Agora, criaremos uma variável do tipo `uniform`, de quatro posições (`vec4`), para receber o dado de cor do nosso programa rodando em CPU.

In [388]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """


# Compilação e Linkagem

### Requisitando slot para a GPU para nossos programas *Vertex* e *Fragment Shaders*

In [389]:
# Request a program and shader slots from GPU
program = glCreateProgram()
vertex = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando nosso código-fonte aos slots solicitados

In [390]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o *Vertex Shader*
Se há algum erro em nosso programa *Vertex Shader*, nosso app para por aqui.

In [391]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o *Fragment Shader*
Se há algum erro em nosso programa *Fragment Shader*, nosso app para por aqui.

In [392]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")


### Associando os programas compilados ao programa principal

In [393]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [394]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')

# Make program the default program
glUseProgram(program)


# Definição de Elementos

### Preparando dados para enviar a GPU
Nesse momento, compilamos nossos *Vertex* e *Fragment Shaders* para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.

In [395]:
def get_vertices_circulo(x_center, y_center, num_vertices, radius):
    circulo = np.zeros(num_vertices, [("position", np.float32, 2)])
    pi = np.pi
    angle = 0.0
    for counter in range(num_vertices):
        angle += 2*pi/num_vertices
        x = math.cos(angle)*radius + x_center
        y = math.sin(angle)*radius + y_center
        circulo[counter] = [x, y]
    return circulo


def get_vertices_pa():
    pa = np.zeros(12, [("position", np.float32, 2)])
    pa['position'] = [
        (0.033, 0.5), (0.033, 0.513), (-0.033, 0.5), (-0.033, 0.513),
        (0.015, 0.1), (0.015, 0.5), (-0.015, 0.1), (-0.015, 0.5),
        (0.1, 0.1), (0.1, 0.0), (-0.1, 0.1), (-0.1, 0.0)
    ]
    pa = np.append(pa, get_vertices_circulo(0, 0, 32, 0.1))
    return pa


def get_vertices_flor5():
    flor = np.zeros(40, [("position", np.float32, 2)])
    flor['position'] = [
        # Pétala 1
        (0, 0), (0.3, -0.1), (0.4, 0), (0.3, 0.1), (0, 0),
        # Pétala 2
        (0, 0), (0.29, 0.15), (0.3, 0.3), (0.15, 0.29), (0, 0),
        # Pétala 3
        (0, 0), (0.1, 0.3), (0, 0.4), (-0.1, 0.3), (0, 0),
        # Pétala 4
        (0, 0), (-0.15, 0.29), (-0.3, 0.3), (-0.29, 0.15), (0, 0),
        # Pétala 5
        (0, 0), (-0.3, 0.1), (-0.4, 0), (-0.3, -0.1), (0, 0),
        # Pétala 6
        (0, 0), (-0.29, -0.15), (-0.3, -0.3), (-0.15, -0.29), (0, 0),
        # Pétala 7
        (0, 0), (-0.1, -0.3), (0, -0.4), (0.1, -0.3), (0, 0),
        # Pétala 8
        (0, 0), (0.15, -0.29), (0.3, -0.3), (0.29, -0.15), (0, 0),
    ]
    # Meio
    flor = np.append(flor, get_vertices_circulo(0, 0, 32, 0.2))
    return flor


In [396]:
# preparando espaço para 3 vértices usando 2 coordenadas (x,y)
vertices = np.zeros(154, [("position", np.float32, 2)])

# preenchendo as coordenadas de cada vértice
vertices['position'] = [
    # -------------- Flor 1 (0 - 21) --------------#
    # Meio
    (-0.01, +0.1), (+0.08, +0.02), (-0.08, 0), (+0.02, -0.07),
    # Petala 1
    (-0.01, +0.1), (+0.16, +0.29), (-0.13, +0.32),
    # Petala 2
    (+0.28, +0.29), (+0.04, +0.05), (+0.31, +0.07),
    # Petala 3
    (+0.33, -0.06), (+0.06, -0.03), (+0.2, -0.28),
    # Petala 4
    (+0.02, -0.07), (-0.07, -0.24), (+0.13, -0.31),
    # Petala 5
    (-0.05, -0.03), (-0.2, -0.25), (-0.34, -0.07),
    # Petala 6
    (-0.36, +0.06), (-0.05, +0.05), (-0.2, +0.2),

    # -------------- Flor 2 (22 - 70) --------------#
    # Meio
    (0, +0.1), (+0.08, +0.05), (+0.08, -0.04), (0, -0.09),
    (-0.08, -0.04), (-0.08, +0.05), (0, +0.1),
    # Petala 1
    (0, +0.1), (0, +0.19), (+0.08, +0.24), (+0.16, +0.19),
    (+0.16, +0.1), (+0.08, +0.05), (0, +0.1),
    # Petala 2
    (+0.08, +0.05), (+0.16, +0.1), (+0.24, +0.06), (+0.24, -0.04),
    (+0.16, -0.09), (+0.08, -0.04), (+0.08, +0.05),
    # Petala 3
    (+0.08, -0.04), (+0.16, -0.09), (+0.16, -0.18), (+0.08, -0.23),
    (0, -0.18), (0, -0.09), (+0.08, -0.04),
    # Petala 4
    (0, -0.09), (0, -0.18), (-0.08, -0.22), (-0.16, -0.18),
    (-0.16, -0.09), (-0.08, -0.04), (0, -0.09),
    # Petala 5
    (-0.08, -0.04), (-0.16, -0.09), (-0.24, -0.04), (-0.24, +0.05),
    (-0.16, +0.1), (-0.08, +0.05), (-0.08, -0.04),
    # Petala 6
    (-0.08, +0.05), (-0.16, +0.1), (-0.16, +0.19), (-0.08, +0.24),
    (0, +0.19), (0, +0.1), (-0.08, +0.05),

    # -------------- Flor 3 (71 - 113) --------------#
    # Meio
    (0, +0.1), (+0.07, +0.06), (+0.07, -0.02), (0, -0.06),
    (-0.07, -0.02), (-0.07, +0.06), (0, +0.1),
    # Petala 1
    (+0.01, +0.12), (+0.03, +0.2), (+0.11, +0.2),
    (+0.14, +0.13), (+0.08, 0.08), (+0.01, +0.12),
    # Petala 2
    (+0.09, +0.06), (+0.17, +0.09), (+0.21, +0.02),
    (+0.17, -0.04), (+0.09, -0.02), (+0.09, +0.06),
    # Petala 3
    (+0.09, -0.04), (+0.15, -0.09), (+0.12, -0.17),
    (+0.03, -0.16), (+0.01, -0.08), (+0.09, -0.04),
    # Petala 4
    (-0.01, -0.08), (-0.03, -0.17), (-0.11, -0.17),
    (-0.15, -0.1), (-0.08, -0.04), (-0.01, -0.08),
    # Petala 5
    (-0.09, -0.02), (-0.18, -0.05), (-0.22, +0.02),
    (-0.17, +0.09), (-0.09, +0.06), (-0.09, -0.02),
    # Petala 6
    (-0.09, +0.07), (-0.15, +0.13), (-0.11, +0.21),
    (-0.03, +0.2), (-0.01, 0.11), (-0.09, +0.07),

    # -------------- Canteiro (114 - 153) --------------#
    # Borda
    (-0.63, -0.63), (-0.63, 0.63), (0.63, -0.63), (0.63, 0.63),
    # Fundo
    (-0.6, -0.6), (-0.6, 0.6), (0.6, -0.6), (0.6, 0.6),
    # Padrão - 8 quadrados
    (-0.6, -0.6), (-0.6, -0.3), (-0.3, -0.6), (-0.3, -0.3),
    (-0.6, 0.0), (-0.6, 0.3), (-0.3, 0.0), (-0.3, 0.3),
    (-0.3, -0.3), (-0.3, 0.0), (0.0, -0.3), (0.0, 0.0),
    (-0.3, 0.3), (-0.3, 0.6), (0.0, 0.3), (0.0, 0.6),
    (0.0, -0.6), (0.0, -0.3), (0.3, -0.6), (0.3, -0.3),
    (0.0, 0.0), (0.0, 0.3), (0.3, 0.0), (0.3, 0.3),
    (0.3, -0.3), (0.3, 0.0), (0.6, -0.3), (0.6, 0.0),
    (0.3, 0.3), (0.3, 0.6), (0.6, 0.3), (0.6, 0.6),

]
# -------------- Flor 4 (154-313) --------------------#

vertices = np.append(vertices, get_vertices_circulo(0.1, 0.1, 32, 0.1))
vertices = np.append(vertices, get_vertices_circulo(0.1, -0.1, 32, 0.1))
vertices = np.append(vertices, get_vertices_circulo(-0.1, -0.1, 32, 0.1))
vertices = np.append(vertices, get_vertices_circulo(-0.1, 0.1, 32, 0.1))
vertices = np.append(vertices, get_vertices_circulo(0, 0, 32, 0.1))

# -------------- Pá (314 - 357) --------------------#
vertices = np.append(vertices, get_vertices_pa())

# -------------- Flor 5 (358 - 429) --------------------#
vertices = np.append(vertices, get_vertices_flor5())

In [397]:
# Adicionar os offsets aqui:
OFFSET_FLOR_1 = 0
OFFSET_FLOR_2 = 22
OFFSET_FLOR_3 = 71
OFFSET_CANTEIRO = 114
OFFSET_FLOR_4 = 154
OFFSET_PA = 314
OFFSET_FLOR_5 = 358

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [398]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Abaixo, enviamos todo o conteúdo da variável `vertices`.
Veja os parâmetros da função `glBufferData` [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [399]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa `GLSL` (*Vertex Shaders*) com nossos dados
Primeiro, definimos o byte inicial e o offset dos dados.

In [400]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

Em seguida, soliciamos à GPU a localização da variável `position` (que guarda coordenadas dos nossos vértices). Definimos essa variável no *Vertex Shader*.

In [401]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável `position` (aqui identificada na posição `loc`).

Outros parâmetros:
* Definimos que possui duas coordenadas.
* Que cada coordenada é do tipo `float` (`GL_FLOAT`).
* Que não se deve normalizar a coordenada (`False`).

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [402]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

###  Aqui, pegamos a localização da variável color (`uniform`) para que possamos alterá-la em nosso laço da janela.

In [403]:
loc_color = glGetUniformLocation(program, "color")
R = 1.0
G = 0.0
B = 0.0

### Capturando eventos de teclado e modificando variáveis para a matriz de transformação

In [404]:
transl = np.array([[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], dtype=float)
scale = np.array([[0.4, 0.5, 0.5, 0.5, 0.4], [
                 0.4, 0.5, 0.5, 0.5, 0.4]], dtype=float)
theta = np.array([0, 0, 0, 0, 0], dtype=float)

flor_plantada = np.array([False, False, False, False, False])

flor_atual = 0
fim_de_jogo = False

def key_event(window, key, scancode, action, mods):
    global transl, scale, theta, flor_atual, flor_plantada

    # print('[key event] key=', key)
    # print('[key event] scancode=', scancode)
    # print('[key event] action=', action)
    # print('[key event] mods=', mods)

    # Transladar com as setas
    if key == 265 and not flor_plantada[flor_atual]:  # cima
        transl[1][flor_atual] += 0.01
    if key == 264 and not flor_plantada[flor_atual]:  # baixo
        transl[1][flor_atual] -= 0.01
    if key == 263 and not flor_plantada[flor_atual]:  # esquerda
        transl[0][flor_atual] -= 0.01
    if key == 262 and not flor_plantada[flor_atual]:  # direita
        transl[0][flor_atual] += 0.01

    # Escalar com as teclas + e -
    if (key == 334 or key == 61) and scale[0][flor_atual] < 1 and not flor_plantada[flor_atual]:  # aumenta
        scale[0][flor_atual] += 0.01
        scale[1][flor_atual] += 0.01
    if (key == 333 or key == 45) and scale[0][flor_atual] > 0.15 and not flor_plantada[flor_atual]:  # diminui
        scale[0][flor_atual] -= 0.01
        scale[1][flor_atual] -= 0.01

    # Rotacionar com as teclas < e >
    if key == 44 and not flor_plantada[flor_atual]:  # antihorário
        theta[flor_atual] += 0.1
    if key == 46 and not flor_plantada[flor_atual]:  # horário
        theta[flor_atual] -= 0.1

    # Alterar entre as flores, as teclas 1 - 4
    if key == 49 or key == 321:
        flor_atual = 0
    elif key == 50 or key == 322:
        flor_atual = 1
    elif key == 51 or key == 323:
        flor_atual = 2
    elif key == 52 or key == 324:
        flor_atual = 3
    elif key == 53 or key == 325:
        flor_atual = 4

    # Fixar flores na tela
    if key == 257:  # enter
        flor_plantada[flor_atual] = True


glfw.set_key_callback(window, key_event)


Criaremos uma função genérica para gerar a matriz de transformação das flores de acordo com as variáveis modificadas pelos eventos do teclado:

In [405]:
def monta_matriz(index=flor_atual, transladar=True, rotacionar=True, escalar=True):
    t_x = transl[0][index] if transladar else 1
    t_y = transl[1][index] if transladar else 1
    s_x = scale[0][index] if escalar else 1
    s_y = scale[1][index] if escalar else 1
    th = theta[index] if rotacionar else 1

    return np.array([math.cos(th)*s_x, -math.sin(th)*s_y, 0.0, t_x,
                     math.sin(th)*s_x, math.cos(th)*s_y, 0.0, t_y,
                     0.0, 0.0, 1.0, 0.0,
                     0.0, 0.0, 0.0, 1.0], np.float32)


### Nesse momento, exibimos a janela.

In [406]:
glfw.show_window(window)


# Execução

### Objetos
Alguns objetos serão mostrados na janela, e para isso os definimos em diferentes funções.

In [407]:
def desenhar_canteiro(program):
    matriz_canteiro = np.array([1.0, 0.0, 0.0, 0.0,
                             0.0, 1.0, 0.0, 0.0,
                             0.0, 0.0, 1.0, 0.0,
                             0.0, 0.0, 0.0, 1.0], np.float32)
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_canteiro)

    offset = OFFSET_CANTEIRO
    # Borda
    glUniform4f(loc_color, 0.20, 0.11, 0.03, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset += 4
    # Fundo
    glUniform4f(loc_color, 0.38, 0.22, 0.07, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset += 4
    # Padrao
    for i in range(8):
        glUniform4f(loc_color, 0.38, 0.26, 0.07, 1.0)
        glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
        offset += 4

def desenhar_flor0():
    offset = OFFSET_FLOR_1
    # Meio (Amarelo)
    glUniform4f(loc_color, 1, 1, 0, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset += 4

    # Petalas (Brancas)
    for i in range(6):
        glUniform4f(loc_color, 1, 1, 1, 1.0)
        glDrawArrays(GL_TRIANGLE_STRIP, offset, 3)
        offset += 3


def desenhar_flor1():
    offset = OFFSET_FLOR_2
    glUniform4f(loc_color, 0.41, 0.35, 0.80, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 7)
    offset += 7

    for i in range(3):
        glUniform4f(loc_color, 0.28, 0.24, 0.54, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, offset, 7)
        offset += 7
        glUniform4f(loc_color, 1, 1, 1, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, offset, 7)
        offset += 7


def desenhar_flor2():
    offset = OFFSET_FLOR_3
    glUniform4f(loc_color, 0.87, 0.19, 0.39, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 7)
    offset += 7

    for i in range(6):
        glUniform4f(loc_color, 0.87, 0.19, 0.39, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, offset, 6)
        offset += 6


def desenhar_flor3():
    offset = OFFSET_FLOR_4
    for i in range(4):
        glUniform4f(loc_color, 0.87, 0.87, 0.0, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, offset, 32)
        offset += 32

    glUniform4f(loc_color, 0.9, 0.9, 0.9, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 32)


def desenhar_flor4():
    offset = OFFSET_FLOR_5
    for i in range(8):
        glUniform4f(loc_color, 1.0, 0.7, 0.1, 1.0)
        glDrawArrays(GL_TRIANGLE_STRIP, offset, 5)
        offset += 5

    glUniform4f(loc_color, 0.25, 0.12, 0.0, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 32)

def desenhar_flor(program, flor):
    loc = glGetUniformLocation(program, "mat_transformation")
    matriz_flor = monta_matriz(flor)
    glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_flor)

    if flor == 0:
        desenhar_flor0()
    elif flor == 1:
        desenhar_flor1()
    elif flor == 2:
        desenhar_flor2()
    elif flor == 3:
        desenhar_flor3()
    elif flor == 4:
        desenhar_flor4()

def desenhar_pa(program, flor = None, fim_de_jogo=False):
    if fim_de_jogo:
        matriz_pa = np.array([1.0, 0.0, 0.0, 0.8,
                              0.0, 1.0, 0.0, -0.2,
                              0.0, 0.0, 1.0, 0.0,
                              0.0, 0.0, 0.0, 1.0], np.float32)
        glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_pa)
    else:
        matriz_pa = monta_matriz(flor, escalar=False)
        glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_pa)

    offset = OFFSET_PA
    # Cabo da pá
    for i in range(2):
        glUniform4f(loc_color, 0.9, 0.9, 0.9, 1.0)
        glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
        offset += 4
    # Pá
    glUniform4f(loc_color, 0.4, 0.4, 0.4, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset += 4
    glUniform4f(loc_color, 0.4, 0.4, 0.4, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 32)
    offset += 32


### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a `OpenGL`.

A novidade agora é a função `glDrawArrays()`.

Tal função recebe o tipo de primitiva (`GL_TRIANGLES`), o índice inicial do array de vértices (mostraremos todos os três vértices, por isso começamos com $0$) e a quantidade de vértices (`len(vertices)`).

In [408]:
print("Boas vindas ao Jardim do ICMC!")
print("------------------------------")
print("Como jardineiro, você deve plantar todas as 5 flores, utilizando as teclas de 1 a 5 para selecionar uma flor e ENTER para plantar.")
print("Para mover utilize as setas do teclado.")
print("Para escalar utilize as seguintes teclas: '+', '-'.")
print("Para rotacionar utilize as seguintes teclas: '<', '>'.")
print("Ao plantar todas as flores, você terá ajudado na construção do Jardim do ICMC.")
print("------------------------------")

while not glfw.window_should_close(window):
    glfw.poll_events()

    # Definindo a cor de fundo
    glClearColor(0.45, 0.59, 0.23, 1.0)
    glClear(GL_COLOR_BUFFER_BIT)

    desenhar_canteiro(program)

    def desenhar_plantando_flor(flor):
        if flor_atual == flor or flor_plantada[flor]:
            if flor_atual == flor and not flor_plantada[flor]:
                desenhar_pa(program, flor=flor)
            desenhar_flor(program, flor)

    desenhar_plantando_flor(0)
    desenhar_plantando_flor(1)
    desenhar_plantando_flor(2)
    desenhar_plantando_flor(3)
    desenhar_plantando_flor(4)

    if np.sum(flor_plantada) == 5 and fim_de_jogo == False:
        print("Parabéns!!! Que lindo canteiro ^^")
        fim_de_jogo = True

    if fim_de_jogo:
        desenhar_pa(program, fim_de_jogo=True)

    glfw.swap_buffers(window)

glfw.terminate()


Boas vindas ao Jardim do ICMC!
------------------------------
Como jardineiro, você deve plantar todas as 5 flores, utilizando as teclas de 1 a 5 para selecionar uma flor e ENTER para plantar.
Para mover utilize as setas do teclado.
Para escalar utilize as seguintes teclas: '+', '-'.
Para rotacionar utilize as seguintes teclas: '<', '>'.
Ao plantar todas as flores, você terá ajudado na construção do Jardim do ICMC.
------------------------------
Parabéns!!! Que lindo canteiro ^^
